In [ ]:
# Google Colab Setup
import os
import sys
from pathlib import Path

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("Running in local environment")

# Mount Google Drive if in Colab
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set the project root path - MODIFY THIS PATH to match your Google Drive structure
    # Example: if your Thomas-Fermi folder is in "My Drive/Projects/Thomas-Fermi/"
    PROJECT_ROOT = Path('/content/drive/MyDrive/Thomas-Fermi-Folder/Thomas-Fermi')
    
    # Alternative: Upload files directly to Colab (uncomment if not using Drive)
    # from google.colab import files
    # uploaded = files.upload()  # Upload your project as a zip file
    # !unzip -q your_project.zip
    # PROJECT_ROOT = Path('/content/Thomas-Fermi')
    
else:
    # Local environment
    PROJECT_ROOT = Path('.')

# Change to project directory
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

# Add project root to Python path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Install required packages if in Colab
if IN_COLAB:
    import subprocess
    subprocess.run(['pip', 'install', 'scipy', 'matplotlib', 'numpy'], check=True)

# Core imports and dynamic plotting module loading
import importlib.util, time, re, numpy as np
from datetime import datetime
from dataclasses import asdict

# Import solver modules
try:
    from solvers.solver3_movie import SimulationConfig, ThomasFermiSolver
    print("Successfully imported solver modules")
except ImportError as e:
    print(f"Error importing solver modules: {e}")
    print("Make sure the project structure is correct and all files are present")

# Dynamically import all plotting scripts in plot-code/ so they can be used as
# regular Python modules inside this notebook.
plot_dir = PROJECT_ROOT / 'plot-code'
loaded_modules = []
if plot_dir.exists():
    for script_path in plot_dir.glob('*.py'):
        try:
            spec = importlib.util.spec_from_file_location(script_path.stem, script_path)
            if spec is not None and spec.loader is not None:
                mod = importlib.util.module_from_spec(spec)
                sys.modules[script_path.stem] = mod
                spec.loader.exec_module(mod)
                loaded_modules.append(script_path.stem)
        except Exception as e:
            print(f"Failed to load {script_path.stem}: {e}")
    print(f"Loaded plotting modules: {loaded_modules}")
else:
    print(f"plot-code directory not found at {plot_dir}")

# Verify data files exist
data_files = [
    PROJECT_ROOT / 'data/1-data/James2.txt',
    PROJECT_ROOT / 'data/0-data/Exc_data_digitized.csv'
]
for data_file in data_files:
    if data_file.exists():
        print(f"✓ Found: {data_file}")
    else:
        print(f"✗ Missing: {data_file}")
        print("Please ensure all data files are uploaded to the correct location")


In [ ]:
# --- User-configurable parameters (copied from main1_5.py) ---
DESIRED_PAIRS = [
    (-4.0, -1.50), (-3.70, -1.50), (-3.40, -1.50), (-3.10, -1.50),
    (-2.80, -1.50), (-2.50, -1.50), (-2.20, -1.50), (-1.90, -1.50),
    (-1.60, -1.50), (-1.30, -1.50), (-1.00, -1.50), (-0.70, -1.50),
    (-0.40, -1.50), (-0.10, -1.50), (0.20, -1.50), (0.50, -1.50),
    (0.80, -1.50), (1.10, -1.50), (1.40, -1.50), (1.70, -1.50),
    (2.00, -1.50), (2.30, -1.50), (2.60, -1.50), (2.90, -1.50),
    (3.20, -1.50), (3.50, -1.50), (3.80, -1.50)
]

GRID_N = 128

# Optimiser parameters
BASINHOPPING_NITER = 10
BASINHOPPING_STEP_SIZE = 1.0
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2000000

# Potential offset / scaling (empirical)
POTENTIAL_SCALE = 1.0
POTENTIAL_OFFSET = 0.033  # V

# Helper functions -------------------------------------------------------------

def parse_header(james_path: Path):
    """Return mapping idx → (V_QPC, V_SG) from metadata header."""
    header_lines = []
    with james_path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if not line.lstrip().startswith('%'):
                break
            header_lines.append(line.strip())
    header_text = ' '.join(header_lines)
    pat = re.compile(r"@\s*(\d+):\s*VQPC=([+-]?\d*\.?\d+)\s*V,\s*VSG=([+-]?\d*\.?\d+)\s*V", re.I)
    mapping = {}
    for m in pat.finditer(header_text):
        mapping[int(m.group(1)) - 1] = (float(m.group(2)), float(m.group(3)))
    return mapping


def run_single_simulation(idx, x_nm, y_nm, V_vals, pair, batch_dir):
    """Run one simulation (sequential version for notebooks)."""
    cfg = SimulationConfig(
        N=GRID_N,
        potential_data=(x_nm, y_nm, V_vals),
        niter=BASINHOPPING_NITER,
        step_size=BASINHOPPING_STEP_SIZE,
        lbfgs_maxiter=LBFGS_MAXITER,
        lbfgs_maxfun=LBFGS_MAXFUN,
        potential_scale=POTENTIAL_SCALE,
        potential_offset=POTENTIAL_OFFSET,
        exc_file=str(PROJECT_ROOT / 'data/0-data/Exc_data_digitized.csv'),
        solver_type='solver3',
    )

    solver = ThomasFermiSolver(cfg)
    pot_dir = batch_dir / f"pot{idx}"
    pot_dir.mkdir(parents=True, exist_ok=True)

    # 1) Save parameters
    with (pot_dir / 'simulation_parameters.txt').open('w', encoding='utf-8') as f:
        for k, v in asdict(cfg).items():
            f.write(f"{k} = {v}\n")

    # 2) Run optimisation
    t0 = time.time()
    solver.optimise()
    exec_sec = time.time() - t0

    # 3) Save & plot
    title_extra = f"V_QPC={pair[0]:+.2f} V, V_SG={pair[1]:+.2f} V"
    solver.plot_results(save_dir=str(pot_dir), title_extra=title_extra, show=False)
    solver.save_results(pot_dir)

    with (pot_dir / 'simulation_parameters.txt').open('a', encoding='utf-8') as f:
        f.write("\n# Execution time\n")
        f.write(f"execution_time_seconds = {exec_sec:.6f}\n")
        f.write(f"execution_time_minutes = {exec_sec/60:.6f}\n")


def run_batch():
    """Run the full batch of simulations defined in *DESIRED_PAIRS*."""
    # Load combined potential file using absolute path
    data_file = PROJECT_ROOT / 'data/1-data/James2.txt'
    data = np.loadtxt(str(data_file), comments='%')
    mask = (
        (data[:, 0] >= -150) & (data[:, 0] <= 150) &
        (data[:, 1] >= -150) & (data[:, 1] <= 150)
    )
    x_nm = data[mask, 0]
    y_nm = data[mask, 1]
    V_columns = data[mask, 3:]

    idx_to_vs = parse_header(data_file)

    # Map requested pairs → column indices
    idxs = []
    tol = 1e-6
    for pair in DESIRED_PAIRS:
        found = [i for i, vs in idx_to_vs.items() if abs(vs[0]-pair[0]) < tol and abs(vs[1]-pair[1]) < tol]
        if not found:
            raise ValueError(f"Requested pair {pair} not found in header.")
        idxs.append(found[0])

    # Prepare output directories - use absolute path
    base_dir = PROJECT_ROOT / 'analysis_folder'
    today = datetime.now().strftime('%Y%m%d')
    date_dir = base_dir / today
    date_dir.mkdir(parents=True, exist_ok=True)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    batch_dir = date_dir / timestamp
    batch_dir.mkdir(parents=True, exist_ok=True)

    # Run sequential simulations
    for idx in idxs:
        V_vals = V_columns[:, idx]
        pair = idx_to_vs[idx]
        print(f"Starting simulation idx={idx} pair={pair}")
        run_single_simulation(idx, x_nm, y_nm, V_vals, pair, batch_dir)
        print(f"Finished simulation idx={idx}")

    print(f"All simulations complete. Results stored in {batch_dir}")
    return batch_dir

# Uncomment the next line to run immediately
# batch_dir = run_batch()


In [ ]:
# Example: Plot centre density map
# Uncomment and modify the path below after running simulations
# import plot_density_center_2d as pmap
# run_dir = PROJECT_ROOT / 'analysis_folder/20250702/run_152740'  # Update with your actual run directory
# james_file = PROJECT_ROOT / 'data/1-data/James2.txt'
# pmap.plot_density_2d(run_dir, james_file)

# Example: Plot linecuts
# import plot_linecut_x as plx
# results_file = PROJECT_ROOT / 'analysis_folder/20250702/run_152740/pot0/results.npz'  # Update path
# plx.plot_x_linecut(results_file)

# Example: Plot energy decrease  
# import plot_energy_decrease as ped
# results_file = PROJECT_ROOT / 'analysis_folder/20250702/run_152740/pot0/results.npz'  # Update path
# ped.plot_energy_decrease(results_file)

# Helper function to run simulations and create plots
def run_and_plot():
    """Run simulations and create example plots."""
    # Run simulations
    batch_dir = run_batch()
    
    # Create center density map
    if 'plot_density_center_2d' in loaded_modules:
        import plot_density_center_2d as pmap
        james_file = PROJECT_ROOT / 'data/1-data/James2.txt'
        pmap.plot_density_2d(batch_dir, james_file)
    
    # Create linecut for first simulation
    pot_dirs = list(batch_dir.glob('pot*'))
    if pot_dirs and 'plot_linecut_x' in loaded_modules:
        import plot_linecut_x as plx
        results_file = pot_dirs[0] / 'results.npz'
        if results_file.exists():
            plx.plot_x_linecut(results_file)
    
    return batch_dir

# Uncomment to run simulations and plots automatically
# batch_dir = run_and_plot()


In [ ]:
# Core imports and dynamic plotting module loading
import importlib.util, sys, time, re, numpy as np
from pathlib import Path
from datetime import datetime
from dataclasses import asdict

from solvers.solver3_movie import SimulationConfig, ThomasFermiSolver

# Dynamically import all plotting scripts in plot-code/ so they can be used as
# regular Python modules inside this notebook.
plot_dir = Path('plot-code')
loaded_modules = []
for script_path in plot_dir.glob('*.py'):
    spec = importlib.util.spec_from_file_location(script_path.stem, script_path)
    if spec is not None and spec.loader is not None:
        mod = importlib.util.module_from_spec(spec)
        sys.modules[script_path.stem] = mod
        spec.loader.exec_module(mod)
        loaded_modules.append(script_path.stem)
print(f"Loaded plotting modules: {loaded_modules}")


In [ ]:
# --- User-configurable parameters (copied from main1_5.py) ---
DESIRED_PAIRS = [
    (-4.0, -1.50), (-3.70, -1.50), (-3.40, -1.50), (-3.10, -1.50),
    (-2.80, -1.50), (-2.50, -1.50), (-2.20, -1.50), (-1.90, -1.50),
    (-1.60, -1.50), (-1.30, -1.50), (-1.00, -1.50), (-0.70, -1.50),
    (-0.40, -1.50), (-0.10, -1.50), (0.20, -1.50), (0.50, -1.50),
    (0.80, -1.50), (1.10, -1.50), (1.40, -1.50), (1.70, -1.50),
    (2.00, -1.50), (2.30, -1.50), (2.60, -1.50), (2.90, -1.50),
    (3.20, -1.50), (3.50, -1.50), (3.80, -1.50)
]

GRID_N = 128

# Optimiser parameters
BASINHOPPING_NITER = 10
BASINHOPPING_STEP_SIZE = 1.0
LBFGS_MAXITER = 1000
LBFGS_MAXFUN = 2000000

# Potential offset / scaling (empirical)
POTENTIAL_SCALE = 1.0
POTENTIAL_OFFSET = 0.033  # V

# Helper functions -------------------------------------------------------------

def parse_header(james_path: Path):
    """Return mapping idx → (V_QPC, V_SG) from metadata header."""
    header_lines = []
    with james_path.open('r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            if not line.lstrip().startswith('%'):
                break
            header_lines.append(line.strip())
    header_text = ' '.join(header_lines)
    pat = re.compile(r"@\s*(\d+):\s*VQPC=([+-]?\d*\.?\d+)\s*V,\s*VSG=([+-]?\d*\.?\d+)\s*V", re.I)
    mapping = {}
    for m in pat.finditer(header_text):
        mapping[int(m.group(1)) - 1] = (float(m.group(2)), float(m.group(3)))
    return mapping


def run_single_simulation(idx, x_nm, y_nm, V_vals, pair, batch_dir):
    """Run one simulation (sequential version for notebooks)."""
    cfg = SimulationConfig(
        N=GRID_N,
        potential_data=(x_nm, y_nm, V_vals),
        niter=BASINHOPPING_NITER,
        step_size=BASINHOPPING_STEP_SIZE,
        lbfgs_maxiter=LBFGS_MAXITER,
        lbfgs_maxfun=LBFGS_MAXFUN,
        potential_scale=POTENTIAL_SCALE,
        potential_offset=POTENTIAL_OFFSET,
        exc_file='data/0-data/Exc_data_digitized.csv',
        solver_type='solver3',
    )

    solver = ThomasFermiSolver(cfg)
    pot_dir = batch_dir / f"pot{idx}"
    pot_dir.mkdir(parents=True, exist_ok=True)

    # 1) Save parameters
    with (pot_dir / 'simulation_parameters.txt').open('w', encoding='utf-8') as f:
        for k, v in asdict(cfg).items():
            f.write(f"{k} = {v}\n")

    # 2) Run optimisation
    t0 = time.time()
    solver.optimise()
    exec_sec = time.time() - t0

    # 3) Save & plot
    title_extra = f"V_QPC={pair[0]:+.2f} V, V_SG={pair[1]:+.2f} V"
    solver.plot_results(save_dir=str(pot_dir), title_extra=title_extra, show=False)
    solver.save_results(pot_dir)

    with (pot_dir / 'simulation_parameters.txt').open('a', encoding='utf-8') as f:
        f.write("\n# Execution time\n")
        f.write(f"execution_time_seconds = {exec_sec:.6f}\n")
        f.write(f"execution_time_minutes = {exec_sec/60:.6f}\n")


def run_batch():
    """Run the full batch of simulations defined in *DESIRED_PAIRS*."""
    # Load combined potential file
    data = np.loadtxt('data/1-data/James2.txt', comments='%')
    mask = (
        (data[:, 0] >= -150) & (data[:, 0] <= 150) &
        (data[:, 1] >= -150) & (data[:, 1] <= 150)
    )
    x_nm = data[mask, 0]
    y_nm = data[mask, 1]
    V_columns = data[mask, 3:]

    idx_to_vs = parse_header(Path('data/1-data/James2.txt'))

    # Map requested pairs → column indices
    idxs = []
    tol = 1e-6
    for pair in DESIRED_PAIRS:
        found = [i for i, vs in idx_to_vs.items() if abs(vs[0]-pair[0]) < tol and abs(vs[1]-pair[1]) < tol]
        if not found:
            raise ValueError(f"Requested pair {pair} not found in header.")
        idxs.append(found[0])

    # Prepare output directories
    base_dir = Path('analysis_folder')
    today = datetime.now().strftime('%Y%m%d')
    date_dir = base_dir / today
    date_dir.mkdir(parents=True, exist_ok=True)

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    batch_dir = date_dir / timestamp
    batch_dir.mkdir(parents=True, exist_ok=True)

    # Run sequential simulations
    for idx in idxs:
        V_vals = V_columns[:, idx]
        pair = idx_to_vs[idx]
        print(f"Starting simulation idx={idx} pair={pair}")
        run_single_simulation(idx, x_nm, y_nm, V_vals, pair, batch_dir)
        print(f"Finished simulation idx={idx}")

    print(f"All simulations complete. Results stored in {batch_dir}")
    return batch_dir

# Uncomment the next line to run immediately
# batch_dir = run_batch()
